In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Carregar os dados
dados = pd.read_csv('animes.csv')

# Tratamento de valores nulos
for col in dados.columns:
    if dados[col].dtype in ["float64", "int64"]:
        dados[col].fillna(dados[col].mean(), inplace=True)
    else:
        dados[col].fillna('Desconhecido', inplace=True)

# Normalização e renomeação de tipos de dados
dados['rate'] = dados['rate'].astype(float)
dados.rename(columns={
    'anime': 'Nome do Anime',
    'episodes': 'Número de Episódios',
    'votes': 'Votos',
    'rate': 'Classificação'
}, inplace=True)

# Criação de novas colunas
bins = [0, 2, 4, 6, 8, 10]
labels = ['Muito Ruim', 'Ruim', 'Médio', 'Bom', 'Excelente']
dados['Classificação Categórica'] = pd.cut(dados['Classificação'], bins=bins, labels=labels)

# Análise Comparativa por Gênero
genres_columns = [col for col in dados.columns if 'genre_' in col]
genre_summary = dados[genres_columns + ['Votos', 'Classificação']].melt(var_name='Gênero', value_name='Presença')
genre_summary = genre_summary[genre_summary['Presença'] == 1].drop('Presença', axis=1)
genre_stats = genre_summary.groupby('Gênero').agg({'Votos': ['sum', 'mean', 'count'], 'Classificação': ['mean', 'std']}).reset_index()
genre_stats.columns = ['Gênero', 'Votos_Total', 'Votos_Média', 'Animes_por_Gênero', 'Classificação_Média', 'Classificação_Desvio_Padrão']

# Adicionando os dados de gênero ao dataframe principal
for col in genre_stats.columns[1:]:
    dados[col] = dados.apply(lambda x: genre_stats[genre_stats['Gênero'] == x['Gênero']][col].values[0] if x['Gênero'] in genre_stats['Gênero'].values else np.nan, axis=1)

# Correlação entre Número de Episódios e Classificação
correlation_coefficient = dados[['Número de Episódios', 'Classificação']].corr().iloc[0, 1]
dados['Correlação_Episódios_Classificação'] = correlation_coefficient

# Adicionando mais indicadores estatísticos
dados['Média_Votos'] = dados['Votos'].mean()
dados['Desvio_Padrão_Votos'] = dados['Votos'].std()

In [ ]:
dados.to_csv('dados_tratados_para_power_bi.csv', index=False, header=True)
print("Dados exportados com sucesso para 'dados_tratados_para_power_bi.csv'. Prontos para serem usados no Power BI.")